<div class="alert alert-block alert-info" align="center">
    <h1>
        Basics
    </h1>
</div>

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mxnet as mx
import time

from keras.datasets import mnist, fashion_mnist, cifar10
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from matplotlib.legend_handler import HandlerLine2D
from keras.utils import to_categorical
from mxnet import nd, autograd, gluon

In [ ]:
# Classes :
# Conv3x3
class Conv3x3: # A Convolution layer using 3x3 filters.
    def __init__(self, num_filters):
        self.num_filters = num_filters
        self.input = 0
        self.output = 0

        # filters is a 3d array with dimensions (num_filters, 3, 3)
        # We divide by 9 to reduce the variance of our initial values
        self.filters = np.random.randn(num_filters, 3, 3) / 9

    def iterate_regions(self, image):
        '''
        Generates all possible 3x3 image regions using valid padding.
        - image is a 2d numpy array
        '''
        h, w = image.shape

        for i in range(h - 2):
            for j in range(w - 2):
                im_region = image[i:(i + 3), j:(j + 3)]
                yield im_region, i, j                

    def forward(self, input):
        '''
        Performs a forward pass of the conv layer using the given input.
        Returns a 3d numpy array with dimensions (h, w, num_filters).
        - input is a 2d numpy array
        '''
        self.input = input
        if(input.ndim == 2): # First conv we catch the image (2D)
            self.last_input = input
            h, w = input.shape
            output = np.zeros((h - 2, w - 2, self.num_filters))
            for im_region, i, j in self.iterate_regions(input):
                output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))

        else: # Second conv we catch the output of maxpool (3D)
            self.last_input = input
            h, w , filters = input.shape
            output = np.zeros((h - 2, w - 2, self.num_filters))
            temp=np.transpose(input)
            for k in range(0, filters):
                for im_region, i, j in self.iterate_regions(temp[k]):
                    output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))
        
        self.output = output
        return output
    
    def iterate_regions_back(self, image):
        '''
        Generates non-overlapping 2x2 image regions to pool over.
        - image is a 2d numpy array
        '''
        h, w, _ = image.shape
        new_h = h // 2
        new_w = w // 2

        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                yield im_region, i, j

    def backprop(self, d_L_d_out, learn_rate):
        '''
        Performs a backward pass of the conv layer.
        - d_L_d_out is the loss gradient for this layer's outputs.
        - learn_rate is a float.
        '''
        if(self.last_input.ndim == 2): # 1er conv on recup l'image 2D
            d_L_d_filters = np.zeros(self.filters.shape)

            for im_region, i, j in self.iterate_regions(self.last_input):
                for f in range(self.num_filters):
                    d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region

            # Update filters
            self.filters -= learn_rate * d_L_d_filters
            # We aren't returning anything here since we use Conv3x3 as
            # the first layer in our CNN. Otherwise, we'd need to return
            # the loss gradient for this layer's inputs, just like every
            # other layer in our CNN.
            return None

        else:
            d_L_d_input = np.zeros(self.last_input.shape)

            for im_region, i, j in self.iterate_regions_back(self.last_input):
                h, w, f = im_region.shape
                amax = np.amax(im_region, axis=(0, 1))

                for i2 in range(h):
                    for j2 in range(w):
                        for f2 in range(f):
                            # If this pixel was the max value, copy the gradient to it.
                            if im_region[i2, j2, f2] == amax[f2]:
                                d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]
            return d_L_d_input

# ReLU
class Relu:
    def __init__(self):
        self.params = []

    def forward(self, input):
        self.last_input = input
        output = np.maximum(0, input)  # Element-wise
        return output

    def backprop(self, dOut):
        '''
        f′(x) = {1 if x > 0}
                {0 otherwise}
        '''
        dOut[dOut <= 0] = 0
        dOut[dOut > 0] = 1
        
        return dOut

# MaxPool2
class MaxPool2: # A Max Pooling layer using a pool size of 2.
    def iterate_regions(self, image):
        '''
        Generates non-overlapping 2x2 image regions to pool over.
        - image is a 2d numpy array
        '''
        h, w, _ = image.shape
        new_h = h // 2
        new_w = w // 2

        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                yield im_region, i, j

    def forward(self, input):
        '''
        Performs a forward pass of the maxpool layer using the given input.
        Returns a 3d numpy array with dimensions (h / 2, w / 2, num_filters).
        - input is a 3d numpy array with dimensions (h, w, num_filters)
        '''
        self.last_input = input
        h, w, num_filters = input.shape
        output = np.zeros((h // 2, w // 2, num_filters))

        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.amax(im_region, axis=(0, 1))

        return output

    def backprop(self, d_L_d_out):
        '''
        Performs a backward pass of the maxpool layer.
        Returns the loss gradient for this layer's inputs.
        - d_L_d_out is the loss gradient for this layer's outputs.
        '''
        d_L_d_input = np.zeros(self.last_input.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):
            h, w, f = im_region.shape
            amax = np.amax(im_region, axis=(0, 1))

            for i2 in range(h):
                for j2 in range(w):
                    for f2 in range(f):
                        # If this pixel was the max value, copy the gradient to it.
                        if im_region[i2, j2, f2] == amax[f2]:
                            d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]

        return d_L_d_input

# Droupout
class Dropout:
    def __init__(self, drop_probability):
        self.drop_probability = drop_probability
    
    def forward(self, input):
        keep_probability = 1 - self.drop_probability
        mask = nd.random_uniform(0, 1.0, input.shape) < keep_probability
        mask = mask.asnumpy()
        
        if keep_probability > 0.0:
            scale = (1 / keep_probability)
        else:
            scale = 0.0
        out = input * mask * scale
        return out

    def backprop(self, dOut):
        dOut[dOut <= 0] = 0
        dOut[dOut > 0] = 1/ 1 - self.drop_probability 
        
        return dOut

# MyFlatten
class MyFlatten: # A Flattening layer
    def forward(self, input):
        self.last_input_shape = input.shape
        input = input.flatten()
        self.last_input = input
        return input
    
    def backprop(self, d_L_d_out):
        return d_L_d_out.reshape(self.last_input_shape)

# Dense
class Dense: # A standard fully-connected layer with softmax activation.
    def __init__(self, input_len, nodes, **kwargs):
        self.activation_func = kwargs.get("activation", None)
        # We divide by input_len to reduce the variance of our initial values
        self.weights = np.random.randn(input_len, nodes) / input_len
        self.biases = np.zeros(nodes)

    def forward(self, input):
        '''
        Performs a forward pass of the softmax layer using the given input depending on its activation function.
        Returns a 1d numpy array containing the respective probability values.
        - input can be any array with any dimensions.
        '''
        self.last_input_shape = input.shape

        self.last_input = input

        input_len, nodes = self.weights.shape
        
        if(self.activation_func == 'Softmax'):
            output = self.softmax_forward(input)
    
        return output

    def backprop(self, d_L_d_out, learn_rate):
        '''
        Performs a backward pass of the softmax layer depending on its activation function.
        Returns the loss gradient for this layer's inputs.
        - d_L_d_out is the loss gradient for this layer's outputs.
        - learn_rate is a float
        '''
        if(self.activation_func == 'Softmax'):
            output = self.softmax_backprop(d_L_d_out, learn_rate)

        return output

    def softmax_forward(self, input):
        input_len, nodes = self.weights.shape

        totals = np.dot(input, self.weights) + self.biases
        self.last_totals = totals

        exp = np.exp(totals)
        return exp / np.sum(exp, axis=0)

    def softmax_backprop(self, d_L_d_out, learn_rate):
        # We know only 1 element of d_L_d_out will be nonzero
        for i, gradient in enumerate(d_L_d_out):
            if gradient == 0:
                continue

            # e^totals
            t_exp = np.exp(self.last_totals)

            # Sum of all e^totals
            S = np.sum(t_exp)

            # Gradients of out[i] against totals
            d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
            d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)

            # Gradients of totals against weights/biases/input
            d_t_d_w = self.last_input
            d_t_d_b = 1
            d_t_d_inputs = self.weights

            # Gradients of loss against totals
            d_L_d_t = gradient * d_out_d_t

            # Gradients of loss against weights/biases/input
            d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
            d_L_d_b = d_L_d_t * d_t_d_b
            d_L_d_inputs = d_t_d_inputs @ d_L_d_t

            # Update weights / biases
            self.weights -= learn_rate * d_L_d_w
            self.biases -= learn_rate * d_L_d_b
            return d_L_d_inputs

# MyConvolutionalNeuralNetwork
class MyConvolutionalNeuralNetwork:
    def __init__(self):
        self.nbLayers = 0
        self.layers = []
        
    def info(self):
        for i in range(len(self.layers)):
            print(f'\tLayer #{i + 1} => {type(self.layers[i])}')
            if(type(self.layers[i]) is Dense):
                print(f'\t\tThis layer uses the {self.layers[i].activation_func} function as its activation')
        
    def addLayer(self, layer):
        self.nbLayers += 1
        self.layers.append(layer)
        
    def forward_propagation(self, X):
        outPrevious = 0
        for num_layer in range (0, self.nbLayers):
            if(type(self.layers[num_layer]) is Conv3x3):
                if num_layer == 0:
                    X = (X / 255) - 0.5
                    outPrevious = self.layers[num_layer].forward(X)
                else: 
                    outPrevious = self.layers[num_layer].forward(outPrevious)
            
            if(type(self.layers[num_layer]) is Relu):
                outPrevious = self.layers[num_layer].forward(outPrevious)
                      
            if(type(self.layers[num_layer]) is MaxPool2):
                outPrevious = self.layers[num_layer].forward(outPrevious) 
              
            if(type(self.layers[num_layer]) is Dropout):
                outPrevious = self.layers[num_layer].forward(outPrevious)
                  
            if(type(self.layers[num_layer]) is MyFlatten):
                outPrevious = self.layers[num_layer].forward(outPrevious) 
                
            if(type(self.layers[num_layer]) is Dense):
                outPrevious = self.layers[num_layer].forward(outPrevious)
        return outPrevious

    def cost_function(self, out, y):
        return (-np.log(out[y]))

    def backward_propagation(self, out, y, eta):
        previousGradient = 0
        for num_layer in range (self.nbLayers - 1, 0, -1):
            if(type(self.layers[num_layer]) is Dense):
                # Init the gradient at the end
                gradient = np.zeros(10)
                gradient[y] = -1 / out[y]
                previousGradient = self.layers[num_layer].backprop(gradient, eta)
            if(type(self.layers[num_layer]) is MyFlatten):
                previousGradient = self.layers[num_layer].backprop(previousGradient)
            if(type(self.layers[num_layer]) is Dropout):
                previousGradient = self.layers[num_layer].backprop(previousGradient)
            if(type(self.layers[num_layer]) is MaxPool2):
                previousGradient = self.layers[num_layer].backprop(previousGradient)
            if(type(self.layers[num_layer]) is Relu):
                previousGradient = self.layers[num_layer].backprop(previousGradient)
            if(type(self.layers[num_layer]) is Conv3x3):
                previousGradient = self.layers[num_layer].backprop(previousGradient, eta)

    def convert_prob_into_class(self, probs):
        probs = np.copy(probs) # To not to lose props, i.e. y_hat
        probs[probs > 0.5] = 1
        probs[probs <= 0.5] = 0
        return probs

    def accuracy(self, out, y):
        acc = 1 if np.argmax(out) == y else 0
        return acc       

    def predict(self, X):
        outPrevious = self.forward_propagation(X)
        return outPrevious

    def fit(self, X, y, *args, **kwargs):    
        epochs = kwargs.get("epochs", 20)
        verbose = kwargs.get("verbose", False)
        eta = kwargs.get("eta", 0.01)
        cost_history = []
        accuracy_history = []
        t = np.zeros(epochs)
        startTime = time.time()
        for nb_epochs in range(epochs):
            cost_all_images = []
            acc_all_images = []
            cost = 0
            acc = 0
            if(verbose is True):
                print(f'\tRunning epoch {nb_epochs + 1}...')
            for i, (im, label) in enumerate(zip(X, y)):
                # Do a forward pass.
                out = self.forward_propagation(im)
                cost += self.cost_function(out, label)
                cost_all_images.append(self.cost_function(out, label))
                acc += self.accuracy(out, label)
                acc_all_images.append(self.accuracy(out, label))
                self.backward_propagation(out, label, eta)
                if(verbose is True):
                    if i % 100 == 99:
                        endTime = time.time()
                        print(f'\t\t(Past 100 steps) Step {i + 1} : '
                              f'Average Loss : {np.float64(cost / 100):.2f} '
                              f'| Accuracy : {acc}% '
                              f'| Time : {endTime - startTime:.2f}')
                        cost = 0
                        acc = 0
                        startTime = time.time()
            current_cost = np.average(cost_all_images)
            cost_history.append(current_cost)  
            current_acc = np.average(acc_all_images)
            accuracy_history.append(current_acc)
        return cost_history, accuracy_history

In [ ]:
# Functions :
def plot_histories(eta, epochs, cost_history, accuracy_history):
    fig, ax = plt.subplots(figsize = (5, 5))
    ax.set_ylabel(r'$J(\theta)$')
    ax.set_xlabel('Epochs')
    ax.set_title(r"$\eta$ :{}".format(eta))
    line1, = ax.plot(range(epochs), cost_history, label = 'Cost')
    line2, = ax.plot(range(epochs), accuracy_history, label = 'Accuracy')
    plt.legend(handler_map = {line1: HandlerLine2D(numpoints = 4)})

<div class="alert alert-block alert-info" align="center">
    <h1>
        Applications
    </h1>
</div>

<div align="center"><h1> Classification mnist data </h1></div>

In [ ]:
# Loading data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Taking some images
X_train = X_train[0:200]
y_train = y_train[0:200]
X_test = X_test[0:80]
y_test = y_test[0:80]

print(f'In this example we\'ll '
      f'learn on : {X_train.shape[0]} examples'
      f' & test on : {X_test.shape[0]} examples\n')

print(f'Creating the network...')
network = MyConvolutionalNeuralNetwork()
network.addLayer(Conv3x3(8))
network.addLayer(Relu())
network.addLayer(MaxPool2()) 
network.addLayer(Dropout(0.1))
network.addLayer(MyFlatten())
network.addLayer(Dense(13 * 13 * 8, 10, activation = "Softmax"))

# Show some information of the network
# Comment the following line to hide the network's information
network.info()

print(f'\nLearning...')
epochs = 2
# Set verbose to False to hide learning's information
cost_history, accuracy_history = network.fit(X_train, y_train, verbose = True, epochs = epochs)

print(f'\nTesting...')
accuracy_test = []
startTime = time.time()
for i in range(len(X_test)):
    y_pred = network.predict(X_test[i])
    acc_test = network.accuracy(y_pred, y_test[i])
    accuracy_test.append(acc_test)
endTime = time.time()
print(f'\tTest accuracy : {np.average(accuracy_test)}')
print(f'\tTest time for {X_test.shape[0]} image : {endTime - startTime:.2f}')

# History display
eta = 0.01
plot_histories(eta,epochs,cost_history,accuracy_history)

<div align="center"><h1> Classification fashion mnist data </h1></div>

In [ ]:
# Loading data
((X_train, y_train), (X_test, y_test)) = fashion_mnist.load_data()

# Taking some images
X_train = X_train[0:200]
y_train = y_train[0:200]
X_test = X_test[0:80]
y_test = y_test[0:80]

print(f'In this example we\'ll '
      f'learn on : {X_train.shape[0]} examples'
      f' & test on : {X_test.shape[0]} examples\n')

print(f'Creating the network...')
network = MyConvolutionalNeuralNetwork()
network.addLayer(Conv3x3(8))
network.addLayer(Relu())
network.addLayer(MaxPool2())
network.addLayer(Dropout(0.1))
network.addLayer(MyFlatten())
network.addLayer(Dense(13 * 13 * 8, 10, activation = "Softmax"))

# Show some information of the network
# Comment the following line to hide the network's information
network.info()

print(f'\nLearning...')
epochs = 2
# Set verbose to False to hide learning's information
cost_history, accuracy_history = network.fit(X_train, y_train, verbose = True, epochs = epochs)

print(f'\nTesting...')
accuracy_test = []
startTime = time.time()
for i in range(len(X_test)):
    y_pred = network.predict(X_test[i])
    acc_test = network.accuracy(y_pred, y_test[i])
    accuracy_test.append(acc_test)
endTime = time.time()
print(f'\tTest accuracy : {np.average(accuracy_test)}')
print(f'\tTest time for {X_test.shape[0]} image : {endTime - startTime:.2f}')

# History display
eta = 0.01
plot_histories(eta,epochs,cost_history,accuracy_history)

<div align="center"><h1> Classification cifar 10 data </h1></div>

In [ ]:
# The data, split between train and test sets:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Taking some images
X_train = X_train[0:200]
y_train = y_train[0:200]
X_test = X_test[0:80]
y_test = y_test[0:80]

# Editing the images
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print(f'In this example we\'ll '
      f'learn on : {X_train.shape[0]} examples'
      f' & test on : {X_test.shape[0]} examples\n')

print(f'Creating the network...')
network = MyConvolutionalNeuralNetwork()
network.addLayer(Conv3x3(16))# (32,32,16)
network.addLayer(Relu())     # (32,32,16)
network.addLayer(MaxPool2())  # (16,16,8)
network.addLayer(Dropout(0.1)) # (16,16,8)
network.addLayer(MyFlatten())  # (2048)
network.addLayer(Dense(3600, 10, activation = "Softmax"))

# Show some information of the network
# Comment the following line to hide the network's information
network.info()

print(f'\nLearning...')
epochs = 2
# Set verbose to False to hide learning's information
cost_history, accuracy_history = network.fit(X_train, y_train, verbose = True, epochs = epochs)

print(f'\nTesting...')
accuracy_test = []
startTime = time.time()
for i in range(len(X_test)):
    y_pred = network.predict(X_test[i])
    acc_test = network.accuracy(y_pred, y_test[i])
    accuracy_test.append(acc_test)
endTime = time.time()
print(f'\tTest accuracy : {np.average(accuracy_test)}')
print(f'\tTest time for {X_test.shape[0]} image : {endTime - startTime:.2f}')

# History display
eta = 0.01
plot_histories(eta,epochs,cost_history,accuracy_history)